In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge
from math import sqrt
import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
df = pd.read_csv("electricity_prices.csv", na_values=['?'])
df.head()

,DateTime,Holiday,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
0,01/11/2011 00:00,None,0,1,44,1,11,2011,0,315.31,3388.77,49.26,6.0,9.3,600.71,356.0,3159.60,54.32
1,01/11/2011 00:30,None,0,1,44,1,11,2011,1,321.80,3196.66,49.26,6.0,11.1,605.42,317.0,2973.01,54.23
2,01/11/2011 01:00,None,0,1,44,1,11,2011,2,328.57,3060.71,49.10,5.0,11.1,589.97,311.0,2834.00,54.23
3,01/11/2011 01:30,None,0,1,44,1,11,2011,3,335.60,2945.56,48.04,6.0,9.3,585.94,313.0,2725.99,53.47
4,01/11/2011 02:00,None,0,1,44,1,11,2011,4,342.90,2849.34,33.75,6.0,11.1,571.52,346.0,2655.64,39.87


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38014 entries, 0 to 38013
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateTime                38014 non-null  object 
 1   Holiday                 38014 non-null  object 
 2   HolidayFlag             38014 non-null  int64  
 3   DayOfWeek               38014 non-null  int64  
 4   WeekOfYear              38014 non-null  int64  
 5   Day                     38014 non-null  int64  
 6   Month                   38014 non-null  int64  
 7   Year                    38014 non-null  int64  
 8   PeriodOfDay             38014 non-null  int64  
 9   ForecastWindProduction  38009 non-null  float64
 10  SystemLoadEA            38012 non-null  float64
 11  SMPEA                   38012 non-null  float64
 12  ORKTemperature          37719 non-null  float64
 13  ORKWindspeed            37715 non-null  float64
 14  CO2Intensity            38007 non-null

In [5]:
df.isnull().sum()

DateTime                    0
Holiday                     0
HolidayFlag                 0
DayOfWeek                   0
WeekOfYear                  0
Day                         0
Month                       0
Year                        0
PeriodOfDay                 0
ForecastWindProduction      5
SystemLoadEA                2
SMPEA                       2
ORKTemperature            295
ORKWindspeed              299
CO2Intensity                7
ActualWindProduction        5
SystemLoadEP2               2
SMPEP2                      2
dtype: int64

In [6]:
df.shape

(38014, 18)

In [7]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37682 entries, 0 to 38013
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateTime                37682 non-null  object 
 1   Holiday                 37682 non-null  object 
 2   HolidayFlag             37682 non-null  int64  
 3   DayOfWeek               37682 non-null  int64  
 4   WeekOfYear              37682 non-null  int64  
 5   Day                     37682 non-null  int64  
 6   Month                   37682 non-null  int64  
 7   Year                    37682 non-null  int64  
 8   PeriodOfDay             37682 non-null  int64  
 9   ForecastWindProduction  37682 non-null  float64
 10  SystemLoadEA            37682 non-null  float64
 11  SMPEA                   37682 non-null  float64
 12  ORKTemperature          37682 non-null  float64
 13  ORKWindspeed            37682 non-null  float64
 14  CO2Intensity            37682 non-null

In [8]:
df = df.drop(['DateTime'], axis = 1)

In [9]:
df.corr().abs()['SMPEP2'].sort_values(ascending = False)

SMPEP2                    1.000000
SMPEA                     0.618158
SystemLoadEP2             0.517081
SystemLoadEA              0.491096
PeriodOfDay               0.323490
ActualWindProduction      0.083434
ForecastWindProduction    0.079639
DayOfWeek                 0.069625
Year                      0.045456
ORKWindspeed              0.035436
CO2Intensity              0.035055
WeekOfYear                0.015814
Month                     0.014918
Day                       0.012801
ORKTemperature            0.009087
HolidayFlag               0.001838
Name: SMPEP2, dtype: float64

In [10]:
x = df[['ActualWindProduction', 'SystemLoadEP2', 'SMPEA', 'SystemLoadEA', 'ForecastWindProduction', 
     'DayOfWeek', 'Year', 'ORKWindspeed', 'CO2Intensity', 'PeriodOfDay']]
y = df['SMPEP2']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
model = keras.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=[10]),
        keras.layers.Dense(800, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1024, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation = 'linear'),
        ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5632      
                                                                 
 dense_1 (Dense)             (None, 800)               410400    
                                                                 
 dropout (Dropout)           (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              820224    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 1,237,281
Trainable params: 1,237,281
Non-

In [14]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [15]:
%%time
early_stopping = keras.callbacks.EarlyStopping(patience = 10, min_delta = 0.001, 
                                               restore_best_weights =True )
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=50,
    epochs=500,
    callbacks=[early_stopping],
    verbose=1, 
)

Epoch 1/500
679/679 [==============================] - 10s 14ms/step - loss: 813.2113 - mse: 813.2113 - mae: 16.6094 - val_loss: 680.1269 - val_mse: 680.1269 - val_mae: 13.8848
Epoch 2/500
679/679 [==============================] - 10s 15ms/step - loss: 707.1049 - mse: 707.1049 - mae: 14.8576 - val_loss: 639.3884 - val_mse: 639.3884 - val_mae: 13.0029
Epoch 3/500
679/679 [==============================] - 12s 17ms/step - loss: 701.2467 - mse: 701.2468 - mae: 14.7658 - val_loss: 609.5613 - val_mse: 609.5613 - val_mae: 13.3501
Epoch 4/500
679/679 [==============================] - 12s 17ms/step - loss: 687.3743 - mse: 687.3743 - mae: 14.5525 - val_loss: 611.0126 - val_mse: 611.0126 - val_mae: 13.1251
Epoch 5/500
679/679 [==============================] - 10s 15ms/step - loss: 682.3052 - mse: 682.3052 - mae: 14.4507 - val_loss: 617.7639 - val_mse: 617.7639 - val_mae: 13.1890
Epoch 6/500
679/679 [==============================] - 12s 17ms/step - loss: 679.1365 - mse: 679.1365 - mae: 14.454

679/679 [==============================] - 10s 15ms/step - loss: 499.2906 - mse: 499.2906 - mae: 11.5402 - val_loss: 543.9046 - val_mse: 543.9046 - val_mae: 11.2886
Epoch 47/500
679/679 [==============================] - 10s 15ms/step - loss: 493.3296 - mse: 493.3296 - mae: 11.5170 - val_loss: 537.1215 - val_mse: 537.1215 - val_mae: 11.5677
Epoch 48/500
679/679 [==============================] - 10s 14ms/step - loss: 482.4256 - mse: 482.4256 - mae: 11.3881 - val_loss: 545.8820 - val_mse: 545.8820 - val_mae: 11.2682
Epoch 49/500
679/679 [==============================] - 10s 15ms/step - loss: 490.2566 - mse: 490.2566 - mae: 11.4879 - val_loss: 543.0308 - val_mse: 543.0308 - val_mae: 11.2377
Wall time: 8min 51s


In [16]:
%%time
predictions = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, predictions)}")
print(f"R2_score: {r2_score(y_test, predictions)}")

MAE: 11.347880076840816
R2_score: 0.5643502519173749
Wall time: 555 ms


In [17]:
%%time
from xgboost import XGBRegressor
model2 = XGBRegressor(n_estimators = 8000, max_depth=17, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model2.fit(X_train, y_train)
pred = model2.predict(X_test)
r2_score(y_test, pred)

Wall time: 2min 20s


0.6187556535493742

In [18]:
%%time
model3 = XGBRegressor(n_estimators = 10000, max_depth=17, eta=0.1, subsample=0.7, colsample_bytree=0.8, nthread=4)
model3.fit(X_train, y_train)
pred = model3.predict(X_test)
r2_score(y_test, pred)

Wall time: 2min 7s


0.6187556536406225

In [19]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix


# Split the dataset into features (X) and labels (y)
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), random_state=42, activation='relu')
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9627172615098846
Wall time: 17.7 s


In [17]:
pred[:5]  #xgboost

array([37.34347 , 58.048763, 70.49032 , 62.104763, 47.734825],
      dtype=float32)

In [18]:
predictions[:5]  #keras

array([[ 0.01068195],
       [-0.00647821],
       [ 0.05537845],
       [ 0.02341356],
       [ 0.00585762]], dtype=float32)